API docs available here: https://developers.themoviedb.org/3

In [ ]:
# TODO


# Initialise

In [1]:
import pandas as pd
import requests
import numpy as np
import config
api_key = config.api_key

import dill
import plotly.express as px
import cpi
# cpi.update()

In [2]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [249]:
def list_of_films(start_date, end_date):
    """Query TMDb for movies between two dates.
    
    Will run requests of TMDb API for movies with US theatrical release dates
    between specified dates.  Dates should be given in YYYY-MM-DD format.
    
    Filter out adult movies.  Filter movies that received fewer than 50 votes
    to try and get more populat releases.  Results will be ordered by average
    voter score.  Also filter out documentaries, as we're only interested in
    feature films.
    
    First pull number of pages.  Then cycle through all pages and retrieve 
    full data.
    
    Returns a list of dictionaries.  Each dictionary is one film.
    
    Note that '&with_release_type=3&region=US' selects only US theatrical 
    releases, '&vote_count.gte=50' filters films with fewer than 50 votes,
    and '&without_genres=99|10770' filters out documentaries and TV movies.
    """
    
    query_string = 'https://api.themoviedb.org/3/discover/movie?api_key=' \
                    + api_key \
                    + '&primary_release_date.gte=' + start_date \
                    + '&primary_release_date.lte=' + end_date \
                    + '&include_adult=false' \
                    + '&with_release_type=3&region=US' \
                    + '&sort_by=vote_average.desc' \
                    + '&vote_count.gte=50' \
                    + '&without_genres=99|10770'
                            
    pages = requests.get(query_string).json()['total_pages']

    films_list = []

    for page in tqdm(range(1, pages+1)):
        response = requests.get(query_string + '&page={}'.format(page))
        films = response.json()['results']
        films_list.extend(films)
        
    return films_list

In [268]:
def get_film_details(films):
    """Query TMDb for details on a list of movies."""
    
    films_list = []

    for film in tqdm(films):
        entry = requests.get('https://api.themoviedb.org/3/movie/'
                               + str(film['id']) 
                               + '?api_key=' + api_key
                               + '&language=en-US'
                               + '&append_to_response=credits'
                            )
        entry = (entry.json())
        films_list += [entry]
    
    return films_list

In [251]:
def get_film_list_details(films):
    """Break a long list of films into smaller chunks and pass each 
    smaller list to get_film_details.
    
    This process avoids querying a list of thousands of films, which
    creates problems and tends to break.
    """
    
    idchunks = [films[x:x + 250] for x in range(0, len(films), 250)]

    filmslist = []
    for ids in tqdm(idchunks):
        results = get_film_details(ids)
        filmslist.extend(results)

    return filmslist

In [252]:
def bin_budget(df):
    """Bin budgets into different buckets."""
    
    bins = [0, 2000000, 5000000, 10000000, 30000000, 
            50000000, 100000000, 250000000, 300000000]
    
    labels = ['0-2M', '2-5M', '5-10M', '10-30M', 
              '30-50M', '50-100M', '100-250M', '250-300M']
    
    df['budget_bin'] = pd.cut(df['budget'], bins, labels=labels)
    
    return df

In [253]:
def build_films_df(films_list):
    """Build a dataframe from the list of TMDb API query results.
    
    The dataframe will add columns for release year and decade, adjust
    budgets and revenues for inflation, and bin budgets into buckets.
    """
    
    df = pd.DataFrame(films_list) \
        .drop(columns=['adult', 'backdrop_path', 'imdb_id', 'homepage', 
                       'overview', 'poster_path', 'tagline', 'video',
                       'belongs_to_collection', 'original_title'])

    df['release_date'] = pd.to_datetime(df['release_date'])

    df['year'] = df['release_date'].dt.year

    df['decade'] = ((df.year)//10)*10
    
    df['budget_adj'] = df[df['year'] != 2019] \
        .apply(lambda x: cpi.inflate(x['budget'], x['year']), axis=1)

    df['revenue_adj'] = df[df['year'] != 2019] \
        .apply(lambda x: cpi.inflate(x['revenue'], x['year']), axis=1)
    
    df['profit'] = df['revenue'] - df['budget']

    df['profit_adj'] = df['revenue_adj'] - df['budget_adj']
    
    df = bin_budget(df)
    
    return df

In [ ]:
# Request for a specific movie:
# requests.get('https://api.themoviedb.org/3/movie/'
#                                + '10994' 
#                                + '?api_key=' + api_key
#                                + '&language=en-US').json()

# Get 1990s films

Find the list of films for the 1990s

In [254]:
films = list_of_films('1990-01-01', '1999-12-31')

Pull the full details on each film

In [ ]:
films_list = get_film_list_details(films)

In [ ]:
df_1990s = build_films_df(films_list)

Pickle result

In [257]:
with open('pickles/df_1990s.pkl', 'wb') as file:
    dill.dump(df_1990s, file)

Unpickle result with the following:

In [ ]:
with open('pickles/df_1990s.pkl', 'rb') as file:
    df_1990s = dill.load(file)

## Plot some results

In [ ]:
df_1990s.columns

In [258]:
fig = px.scatter(df_1990s[df_1990s['budget']>0], x='budget', y='vote_average', hover_name='title')
fig.show()

In [259]:
fig = px.scatter(df_1990s[df_1990s['budget']>0], x='budget', y='revenue', color='year', hover_name='title')
fig.show()

In [260]:
fig = px.box(df_1990s[df_1990s['budget']>0], x='year', y='budget')
fig.show()

In [263]:
fig = px.histogram(df_1990s[df_1990s['vote_count']<=100], x='vote_count', nbins=100, color='title')
fig.show()

# Get 2000s

Find the list of films for the 2000s

In [ ]:
films = list_of_films('2000-01-01', '2009-12-31')

Pull the full details on each film

In [ ]:
films_list = get_film_list_details(films)

In [ ]:
df_2000s = build_films_df(films_list)

Pickle result

In [76]:
with open('pickles/df_2000s.pkl', 'wb') as file:
    dill.dump(df_2000s, file)

Unpickle result with the following:

In [ ]:
with open('pickles/df_2000s.pkl', 'rb') as file:
    df_2000s = dill.load(file)

## Plot some results

In [ ]:
highest_revenue = df_2000s.sort_values('revenue', ascending=False)[0:1000]

In [ ]:
fig = px.box(highest_revenue[highest_revenue['budget']>0], x='year', y='budget',
             hover_data=['title'], points='all')
fig.show()

In [18]:
fig = px.scatter(df_2000s[df_2000s['budget']>0], x='budget', y='revenue', color='year', hover_name='title')
fig.show()

Number of films in budget categories over time?

In [50]:
grouped = df_2000s.groupby(['year', 'budget_bin']).count()

In [53]:
fig = px.line(grouped.reset_index(), x='year', y='id', color='budget_bin')
fig.show()

# Get 2010s

Find the list of films for the 2010s

In [77]:
films = list_of_films('2010-01-01', '2019-12-31')

2010-01-01 2019-12-31
Number of pages =  178


Pull the full details on each film

In [266]:
example = requests.get('https://api.themoviedb.org/3/movie/'
                               + '239459' 
                               + '?api_key=' + api_key
                               + '&language=en-US'
                               + '&append_to_response=credits').json()

In [267]:
example

{'adult': False,
 'backdrop_path': '/mMKahLSpwb9Yj2B0tB6vku3tkGy.jpg',
 'belongs_to_collection': None,
 'budget': 0,
 'genres': [{'id': 99, 'name': 'Documentary'}],
 'homepage': 'http://www.amctv.com/shows/breaking-bad',
 'id': 239459,
 'imdb_id': 'tt3088036',
 'original_language': 'en',
 'original_title': 'No Half Measures: Creating the Final Season of Breaking Bad',
 'overview': 'A documentary about the making of season five of the acclaimed AMC series Breaking Bad.',
 'popularity': 4.659,
 'poster_path': '/8OixSR45U5dbqv8F0tlspmTbXxN.jpg',
 'production_companies': [{'id': 34,
   'logo_path': '/GagSvqWlyPdkFHMfQ3pNq6ix9P.png',
   'name': 'Sony Pictures',
   'origin_country': 'US'}],
 'production_countries': [{'iso_3166_1': 'US',
   'name': 'United States of America'}],
 'release_date': '2013-11-26',
 'revenue': 0,
 'runtime': 135,
 'spoken_languages': [{'iso_639_1': 'en', 'name': 'English'}],
 'status': 'Released',
 'tagline': '',
 'title': 'No Half Measures: Creating the Final Seaso

In [78]:
films_list = get_film_list_details(films)

In [79]:
df_2010s = build_films_df(films_list)

Pickle result

In [80]:
with open('pickles/df_2010s.pkl', 'wb') as file:
    dill.dump(df_2010s, file)

Unpickle result with the following:

In [ ]:
with open('pickles/df_2010s.pkl', 'rb') as file:
    df_2010s = dill.load(file)

## Plot some results

In [58]:
highest_revenue = df_2010s.sort_values('revenue', ascending=False)[0:1000]

In [59]:
fig = px.box(highest_revenue[highest_revenue['budget']>0], x='year', y='budget',
             hover_data=['title'], points='all')
fig.show()

In [ ]:
fig = px.scatter(df_2010s[df_2010s['budget']>0], x='budget', y='revenue', color='year', hover_name='title')
fig.show()

In [ ]:
grouped = df_2010s.groupby(['year', 'budget_bin']).count()

In [ ]:
fig = px.line(grouped.reset_index(), x='year', y='id', color='budget_bin')
fig.show()

# Check all decades

In [213]:
all_films = pd.concat([df_1990s, df_2000s, df_2010s], axis=0, sort=False)

In [210]:
genres = all_films['genres'].apply(pd.Series) \
            .applymap(lambda x: x.get('name', np.nan) \
                  if isinstance(x, dict) else np.nan)

all_films = all_films.merge(genres, left_index=True, right_index=True)

In [211]:
genres[1].value_counts()

Drama              1053
Thriller            689
Comedy              632
Romance             501
Adventure           376
Action              344
Crime               302
Horror              299
Family              265
Fantasy             203
Science Fiction     191
Animation           175
Mystery             171
History             119
Music                85
War                  44
Western              16
TV Movie              8
Name: 1, dtype: int64

In [87]:
all_films = all_films[(all_films[0] != 'Documentary|TV Movie')
                    & (all_films[1] != 'Documentary|TV Movie')
                    & (all_films[2] != 'Documentary|TV Movie')
                    & (all_films[3] != 'Documentary|TV Movie')
                    & (all_films[4] != 'Documentary|TV Movie')
                    & (all_films[5] != 'Documentary|TV Movie')
                    & (all_films[6] != 'Documentary|TV Movie')
                    & (all_films[7] != 'Documentary|TV Movie')
                    & (all_films[8] != 'Documentary|TV Movie')] \
            .drop(columns=[0,1,2,3,4,5,6,7,8]) \
            .reset_index(drop=True)

In [89]:
bins = [1, 2000000, 5000000, 10000000, 30000000, 50000000, 100000000, 
        250000000, 300000000]
labels = ['<2M', '2-5M', '5-10M', '10-30M', '30-50M', '50-100M', '100-250M', '250-300M']

all_films['budget_adj_bin'] = pd.cut(all_films['budget_adj'], bins, labels=labels)

In [90]:
grouped = all_films.groupby(['year', 'budget_adj_bin']).count()

In [91]:
fig = px.line(grouped.reset_index(), x='year', y='id', color='budget_adj_bin')
fig.show()

In [ ]:
fig = px.box(all_films[all_films['budget']>0], x='year', y='budget_adj')
fig.show()

In [95]:
all_films.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'original_title', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'title', 'video', 'vote_average',
       'vote_count', 'year', 'decade', 'budget_adj', 'revenue_adj',
       'budget_bin', 'budget_adj_bin'],
      dtype='object')

In [114]:
fig = px.parallel_categories(all_films.dropna(subset=['budget_adj_bin']), 
                             dimensions=['budget_adj_bin', 'decade'])

fig.show()

In [ ]:
fig = px.parallel_categories(tips, dimensions=['sex', 'smoker', 'day'], 
                color="size", color_continuous_scale=px.colors.sequential.Inferno,
                labels={'sex':'Payer sex', 'smoker':'Smokers at the table', 'day':'Day of week'})
fig.show()

## Pickle result

In [92]:
with open('pickles/all_films.pkl', 'wb') as file:
    dill.dump(all_films, file)

# Compare Scorsese to the Coen Bros

Find the person codes for Martin Scorsese and the Coen brothers. We'll use Ethan, just because we need one of them.

In [ ]:
response = requests.get('https://api.themoviedb.org/3/search/person?api_key=' 
                            +  api_key 
                            + '&include_adult=false' # filter out adult films
                            + '&language=en-US'
                            + '&query=ethan%coen'
                           ).json()
response

In [ ]:
directors = {'scorsese': '1032',
             'coen': '1224'}

## Martin Scorsese data

In [ ]:
def director_request(director):
    response = requests.get('https://api.themoviedb.org/3/person/'
                            + director + '/'
                            + 'movie_credits/'        
                            + '?api_key=' + api_key
                            + '&language=en-US')
    
    director_df = pd.DataFrame((response.json())['crew']) \
                    .drop(columns=['adult', 
                                   'backdrop_path',
                                   'poster_path',
                                   'credit_id',])
    
    return director_df

In [ ]:
def films_list_df(list_of_films):
    
    films_list = []

    for film in tqdm(list_of_films):
        entry = requests.get('https://api.themoviedb.org/3/movie/'
                                   + str(film) 
                                   + '?api_key=' + api_key
                                   + '&language=en-US')
        entry = (entry.json())
        films_list += [entry]
        
    df = pd.DataFrame(films_list) \
            .drop(columns=['adult', 'backdrop_path', 'imdb_id',
                           'homepage', 'overview',
                           'poster_path', 'tagline'])
    
    df['release_date'] = pd.to_datetime(df['release_date'])
    
    df = df[df['status'] == 'Released']
    
    df['year'] = df['release_date'].dt.year

    df['decade'] = ((df.year)//10)*10
    
    df['budget_adj'] = df[(df['year'] != 2019) & (df['status'] == 'Released')] \
        .apply(lambda x: cpi.inflate(x['budget'], x['year']), axis=1)

    df['revenue_adj'] = df[df['year'] != 2019] \
        .apply(lambda x: cpi.inflate(x['revenue'], x['year']), axis=1)
    
    return df

In [ ]:
scorsese = director_request(directors['scorsese'])

We want the list of films Scorsese directed. First filter those out of the dataframe, then get the list of ids.

In [ ]:
scorsese_list = scorsese[scorsese['job'] == 'Director']['id'].to_list()

Run a query for each film in the list.

In [ ]:
scorsese_df = films_list_df(scorsese_list)

In [ ]:
scorsese_df

## Coen Bros data

In [ ]:
coens = director_request(directors['coen'])

In [ ]:
coens_list = coens[coens['job'] == 'Director']['id'].to_list()

In [ ]:
coens_df = films_list_df(coens_list)

## Combine directors

In [ ]:
scorsese_df['director'] = 'Scorsese'

In [ ]:
coens_df['director'] = 'Coens'

In [ ]:
directors = pd.concat([scorsese_df, coens_df]).reset_index(drop=True)

In [ ]:
with open('pickles/scorsese_coens.pkl', 'wb') as file:
    dill.dump(directors, file)

In [ ]:
directors.columns

In [ ]:
directors.sort_values('year', ascending=False)

## Plot some results

In [ ]:
fig = px.scatter(directors[directors['budget']>0], x='budget', y='budget_adj',
                 color='director', hover_name='title')
fig.show()

In [ ]:
fig = px.scatter(directors[directors['budget']>0], x='budget', y='revenue', 
                 color='director', hover_name='title')
fig.show()

In [ ]:
fig = px.box(directors[directors['budget']>0], x='decade', y='budget',
             color='director')
fig.show()

In [ ]:
fig = px.box(directors[directors['budget']>0], x='decade', y='budget_adj',
             color='director')
fig.show()

In [ ]:
fig = px.bar(directors[directors['budget']>0], x='decade', y='budget',
                   color='director', barmode='group')
fig.show()

# Extract genre info

In [216]:
all_films.genres

0                                          [Drama, Crime]
1                                   [Drama, History, War]
2                                 [Fantasy, Drama, Crime]
3                                         [Comedy, Drama]
4                                       [Thriller, Crime]
5                                [Comedy, Drama, Romance]
6                                                 [Drama]
7                                          [Drama, Crime]
8                                                 [Drama]
9                                 [Drama, Music, Romance]
10                               [Thriller, Crime, Drama]
11                             [Crime, Mystery, Thriller]
12                       [Crime, Drama, Thriller, Horror]
13                             [Family, Animation, Drama]
14                               [Drama, Crime, Thriller]
15                                                [Drama]
16                                  [Animation, Thriller]
17            

In [215]:
all_films['genres'] = [[x['name'] for x in list_dict] for list_dict in all_films['genres']]

TypeError: string indices must be integers

In [199]:
df_1990s

,belongs_to_collection,budget,genres,id,original_language,original_title,popularity,production_companies,production_countries,release_date,...,title,video,vote_average,vote_count,year,decade,budget_adj,revenue_adj,budget_bin,profit
0,None,25000000,"[Drama, Crime]",278,en,The Shawshank Redemption,41.988,"[{'id': 97, 'logo_path': '/7znWcbDd4PcJzJUlJxY...","[{'iso_3166_1': 'US', 'name': 'United States o...",1994-09-23,...,The Shawshank Redemption,False,8.7,13843,1994,1990,4.235948e+07,4.802120e+07,10-30M,3341469
1,None,22000000,"[Drama, History, War]",424,en,Schindler's List,28.384,"[{'id': 33, 'logo_path': '/8lvHyhjr8oUKOOy2dKX...","[{'iso_3166_1': 'US', 'name': 'United States o...",1993-11-30,...,Schindler's List,False,8.5,8523,1993,1990,3.823082e+07,5.584577e+08,10-30M,299365567
2,None,60000000,"[Fantasy, Drama, Crime]",497,en,The Green Mile,24.897,"[{'id': 97, 'logo_path': '/7znWcbDd4PcJzJUlJxY...","[{'iso_3166_1': 'US', 'name': 'United States o...",1999-12-10,...,The Green Mile,False,8.5,8803,1999,1990,9.043469e+07,4.289619e+08,50-100M,224600000
3,None,20000000,"[Comedy, Drama]",637,it,La vita è bella,18.689,"[{'id': 370, 'logo_path': None, 'name': 'Melam...","[{'iso_3166_1': 'IT', 'name': 'Italy'}]",1997-12-20,...,Life Is Beautiful,False,8.5,7639,1997,1990,3.129059e+07,3.589031e+08,10-30M,209400000
4,None,8000000,"[Thriller, Crime]",680,en,Pulp Fiction,32.385,"[{'id': 14, 'logo_path': '/m6AHu84oZQxvq7n1rsv...","[{'iso_3166_1': 'US', 'name': 'United States o...",1994-09-10,...,Pulp Fiction,False,8.4,16165,1994,1990,1.355503e+07,3.624764e+08,5-10M,205928762
5,None,55000000,"[Comedy, Drama, Romance]",13,en,Forrest Gump,26.962,"[{'id': 4, 'logo_path': '/fycMZt242LVjagMByZOL...","[{'iso_3166_1': 'US', 'name': 'United States o...",1994-07-06,...,Forrest Gump,False,8.4,15714,1994,1990,9.319086e+07,1.148697e+09,50-100M,622945399
6,None,63000000,[Drama],550,en,Fight Club,31.564,"[{'id': 508, 'logo_path': '/7PzJdsLGlR7oW4J0J5...","[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...",1999-10-15,...,Fight Club,False,8.4,17096,1999,1990,9.495643e+07,1.520113e+08,50-100M,37853753
7,None,25000000,"[Drama, Crime]",769,en,GoodFellas,31.125,"[{'id': 8880, 'logo_path': '/fE7LBw7Jz8R29EABF...","[{'iso_3166_1': 'US', 'name': 'United States o...",1990-09-12,...,GoodFellas,False,8.4,5975,1990,1990,4.803118e+07,8.998429e+07,10-30M,21836394
8,None,20000000,[Drama],73,en,American History X,20.395,"[{'id': 12, 'logo_path': '/iaYpEp3LQmb8AfAtmTv...","[{'iso_3166_1': 'US', 'name': 'United States o...",1998-10-30,...,American History X,False,8.4,6259,1998,1990,3.081067e+07,3.678044e+07,10-30M,3875127
9,None,9000000,"[Drama, Music, Romance]",10376,it,La leggenda del pianista sull'oceano,8.126,"[{'id': 6246, 'logo_path': None, 'name': 'Medu...","[{'iso_3166_1': 'IT', 'name': 'Italy'}]",1998-10-28,...,The Legend of 1900,False,8.3,1193,1998,1990,1.386480e+07,0.000000e+00,5-10M,-9000000


In [233]:
genres = df_1990s['genres'].apply(pd.Series) \
    .merge(df_1990s, left_index = True, right_index = True) \
    .drop(columns=['belongs_to_collection', 'genres', 'video']) \
    .melt(id_vars=['budget', 'id', 'original_language',
       'original_title', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'title', 'vote_average',
       'vote_count', 'year', 'decade', 'budget_adj', 'revenue_adj',
       'budget_bin', 'profit'], value_name='genre') \
    .drop(columns="variable") \
    .dropna(subset=['genre'])

In [203]:
genres.groupby(['year', 'genre'])['budget_adj'].sum().reset_index()

,year,genre,budget_adj
0,1990,Action,1.124000e+09
1,1990,Adventure,7.962244e+08
2,1990,Animation,7.287482e+07
3,1990,Comedy,1.078557e+09
4,1990,Crime,6.945308e+08
5,1990,Drama,1.337040e+09
6,1990,Family,2.717239e+08
7,1990,Fantasy,3.773925e+08
8,1990,History,0.000000e+00
9,1990,Horror,4.082266e+08


In [208]:
fig = px.line(genres.groupby(['year', 'genre'])['profit'].sum().reset_index(),
              x='year', y='profit', color='genre')
fig.show()

In [248]:
fig = px.scatter(genres[(genres['profit']<0) & (genres['revenue'] !=0)], 
                 x='vote_average', y='profit',
                 hover_data=['title','revenue'],
                 color='genre', size='budget')
fig.show()